In [5]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas import DataFrame
import numpy as np

import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

#get the node data
node_data = pd.read_csv ('C:/python/MyScriptsData/Node Example.csv')
node_data.set_index(keys='Organization_Name_Formatted', drop=False, inplace=True)
node_data['Organization_Name_For_Matching'] = node_data['Organization_Name_Formatted']

#remove any nodes that are not in the path finding problem (no delivery/not hub)
node_data = node_data.loc[(node_data['ScheduledForDelivery']==1) | (node_data['Type']=='Hub')]

#get nodes that are in the path finding problem and turn it into a list
orgs_in = node_data['Organization_Name_Formatted']
orgs_in = list(orgs_in)

#print('orgs_in')
#print(orgs_in)

#get distance matrix data
my_distances = pd.read_csv('C:/python/MyScriptsData/Distance Matrix Layout 2.csv',
                          dtype = {"Distance" : "float64"})

#print('my_distances data types')
#print(my_distances.dtypes)

#only get rows for nodes that are in the path finding problem
my_distances = my_distances[my_distances["StartNode"].isin(orgs_in)]

#print('my_distances')
#print(type(my_distances))
#print(my_distances)

#get the hub node
hub_node = node_data.loc[node_data['Type']=='Hub','Organization_Name_Formatted']

#the hub node is node 0
node_dict = {hub_node[0]: 0}
#print('node dictionary')
#print(len(node_data))
dictionary_index = 1
for x in range(len(node_data)):
    #if this is not the hub node (already defined) and this node is in the problem
    if node_data.iloc[x,3] != hub_node[0] and node_data.iloc[x,3] in orgs_in:
        #add to dictionary and increment counter
        node_dict[node_data.iloc[x,3]] = dictionary_index
        dictionary_index = dictionary_index + 1

#print('node_dict')
#print(node_dict)

#create empty list for data structure expected by optimization problem
my_distances_list = []

#for each distance par, put in distance list
for x in range(len(my_distances)):
    #start node = column 0
    my_start_node = node_dict[my_distances.iloc[x,0]]
    #end node = column 1
    my_end_node = node_dict[my_distances.iloc[x,1]]
    #distance between start and end = column 2
    my_link_distance = float(my_distances.iloc[x,2])
    #append to the list
    my_distances_list.append((my_start_node,my_end_node,my_link_distance))

#print('my_distances_list')
#print(my_distances_list)

#get the nearest neighbor starting state
current_node = hub_node[0]
#number of nodes = of nodes in the dictionary
number_nodes = len(node_dict)
#visited node list
visited_nodes = [current_node]
visited_nodes_number = [node_dict[current_node]]
#total cumulative distance
total_distance = 0

#do while there are still nodes to be visited
while len(visited_nodes) < number_nodes:
#    print('current node')
#    print(current_node)
    #set initial value for minimum distance
    minimum_distance = 1000000000000000000000
    next_node = ''
    #go through each row in the distances dataframe
    for x in range(len(my_distances)):
        already_visited = 0
        #if the distance considered in the pair (that is not the current node) is already
        #in the visited array, then skip
        if my_distances.iloc[x,0] == current_node:
            if my_distances.iloc[x,1] in visited_nodes:
                already_visited = 1
        elif my_distances.iloc[x,1] == current_node:
            if my_distances.iloc[x,0] in visited_nodes:
                already_visited = 1
        else: 
            already_visited = 0
        #if the distance pair involves the current node (either in start = column 0 or end = column 1)
        if (my_distances.iloc[x,0] == current_node or my_distances.iloc[x,1] == current_node) \
        and (already_visited==0):
            #print('this distance involves the current node!')
            #print(my_distances.iloc[x,:])
            #if the distance is less than the current minimum distance
            if my_distances.iloc[x,2] < minimum_distance:
                #set the new minimum distance
                minimum_distance = my_distances.iloc[x,2]
                #get the minimum distance node name from this distance row 
                #that is not the current node
                if my_distances.iloc[x,0] == current_node:
                    next_node = my_distances.iloc[x,1]
                else:
                    next_node = my_distances.iloc[x,0]
               # print('this is a new minimum distance!')
    visited_nodes.append(next_node)
    visited_nodes_number.append(node_dict[next_node])
    current_node = next_node
    total_distance = total_distance + minimum_distance
#get distance from last node back to hub
#print('visited nodes ', visited_nodes)
#print('length of visited nodes', len(visited_nodes))
#print('last node', visited_nodes[len(visited_nodes)-1])
#print('type of my_distances ', type(my_distances))
#look for distance pair between hub and last node in the visited path
last_distance = my_distances[(my_distances.StartNode==hub_node[0]) & \
                             (my_distances.EndNode==visited_nodes[len(visited_nodes)-1])]
#if there is no pair for hub and last node, look for last node and hub pair
if len(last_distance)==0:
    last_distance = my_distances[(my_distances.StartNode==visited_nodes[len(visited_nodes)-1]) & \
                             (my_distances.EndNode==hub_node[0])]
#add the last leg to the total distance
total_distance = total_distance + last_distance['Distance'].squeeze()

#set these results as the nearest neighbor results
nearest_neighbor_path = visited_nodes
nearest_neighbor_path_number = visited_nodes_number
nearest_neighbor_path_number = np.array(nearest_neighbor_path_number)
nearest_neighbor_distance = total_distance
#print('nearest_neighbor_path ', nearest_neighbor_path)
#print('nearest neighbor path number', nearest_neighbor_path_number, ' ', type(nearest_neighbor_path_number))
#print('nearest_neighbor_distance ', nearest_neighbor_distance)

#create the fitness function
fitness_dists = mlrose.TravellingSales(distances = my_distances_list)

#create optimization problem object
problem_fit = mlrose.TSPOpt(length = number_nodes, fitness_fn = fitness_dists,maximize=False)

#initially, nearest neighbor is best solution
best_solution_method = 'nearest neighbor'
best_solution_state = nearest_neighbor_path_number
best_solution_distance = nearest_neighbor_distance

#for each of the random optimization techniques, fit and check for the best, hill climb
best_state, best_fitness = mlrose.hill_climb(problem_fit, init_state=np.array(nearest_neighbor_path_number))
#print('here is a best state ,after hill_climb', best_state)
#print('here is a best fitness ,after hill_climb', best_fitness)
#if this is a better solution than current solution, set this as best
if best_fitness < best_solution_distance:
    best_solution = 'hill climb'
    best_solution_state = best_state
    best_solution_distance = best_fitness
#print('solution ', best_solution)
#print('best solution distance ', best_solution_distance)
#print('best solution state ', best_solution_state)
#random hill climb
best_state, best_fitness = mlrose.random_hill_climb(problem_fit, init_state=np.array(nearest_neighbor_path_number))
#print('here is a best state ,after random hill_climb', best_state)
#print('here is a best fitness ,after random hill_climb', best_fitness)
#if this is a better solution than current solution, set this as best
if best_fitness < best_solution_distance:
    best_solution = 'random hill climb'
    best_solution_state = best_state
    best_solution_distance = best_fitness
#print('solution ', best_solution)
#print('best solution distance ', best_solution_distance)
#print('best solution state ', best_solution_state)
#simulated annealing
best_state, best_fitness = mlrose.simulated_annealing(problem_fit, init_state=np.array(nearest_neighbor_path_number))
#print('here is a best state ,after simulated annealing', best_state)
#print('here is a best fitness ,after simulated annealing', best_fitness)
#if this is a better solution than current solution, set this as best
if best_fitness < best_solution_distance:
    best_solution = 'simulated annealing'
    best_solution_state = best_state
    best_solution_distance = best_fitness
#print('solution ', best_solution)
#print('best solution distance ', best_solution_distance)
#print('best solution state ', best_solution_state)

#rearrange the best state list so that the hub is in the first spot
best_state_sorted = []
#start with node = 0 (origin or hub)
next_node = np.where(best_solution_state == 0)
x = int(next_node[0])
#for each element, shift
for i in range(len(best_solution_state)):
    #if have reached the end of the list, reset counter to beginning
    if x >= len(best_solution_state):
        x = 0
    best_state_sorted.append(best_solution_state[x])
    x = x + 1

#print('best state after cut')
#print(best_state_sorted)
        
#put names to the node numbers found by mlrose
#print(len(best_state))
best_state_names = []
for i in range(len(best_state_sorted)):
    for key, value in node_dict.items():
        if best_state_sorted[i] == value:
            best_state_names.append(key)
print('Nearest neighbor state found ', nearest_neighbor_path)
print('The best state found is: ', best_state_names)
print('Nearest neighbor distance found ', nearest_neighbor_distance)
print('The fitness at the best state is: ', best_solution_distance)

#put the list of stops (in order of best path) into a dataframe
print_out = pd.DataFrame(best_state_names,
                   columns=['Organization_Name_Formatted'])
#use index to indicate the visit order number
print_out['StopNumber'] = print_out.index

#merge with the node data to get proper name and address
print_out = print_out.merge(node_data, left_on='Organization_Name_Formatted', right_on='Organization_Name_For_Matching')
#only printing out a subset
print_out = print_out[['StopNumber', 'OrganizationName', 'Address', 'City', 'State',
                      'Zip', 'Phone']]


print_out.to_csv(r'C:\python\MyScriptsOutput\RoutePlan.csv', index = False)



Nearest neighbor state found  ['North_Star_Food_Hub', 'Blue_Mountain_Lake_Hamilton_Helps', 'Minerva_Food_Pantry', 'Westport_Food_Shelf', 'Community_Lunch_Box', 'Bellmont_Food_Pantry', 'Bombay_Food_Pantry', 'Comfort_Food_Community_Greenwich_Food_Pantry']
The best state found is:  ['North_Star_Food_Hub', 'Blue_Mountain_Lake_Hamilton_Helps', 'Minerva_Food_Pantry', 'Comfort_Food_Community_Greenwich_Food_Pantry', 'Westport_Food_Shelf', 'Community_Lunch_Box', 'Bellmont_Food_Pantry', 'Bombay_Food_Pantry']
Nearest neighbor distance found  608.2
The fitness at the best state is:  506.9


In [38]:
node_data

#put the list of stops (in order of best path) into a dataframe
print_out = pd.DataFrame(best_state_names,
                   columns=['Organization_Name_Formatted'])
#use index to indicate the visit order number
print_out['StopNumber'] = print_out.index

#merge with the node data to get proper name and address
print_out = print_out.merge(node_data, left_on='Organization_Name_Formatted', right_on='Organization_Name_For_Matching')
#only printing out a subset
print_out = print_out[['StopNumber', 'OrganizationName']]

#print_out.merge(node_data, left_on='Organizations_Name_Formatted', right_on='Organizations_Name_Formatted')

print_out 

#df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
             #       'value': [1, 2, 3, 5]})
#df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
              #      'value': [5, 6, 7, 8]})
#df1.merge(df2, left_on='lkey', right_on='rkey')


,StopNumber,OrganizationName
0,0,North Star Food Hub
1,1,Blue Mountain Lake Hamilton Helps
2,2,Minerva Food Pantry
3,3,Comfort Food Community (Greenwich Food Pantry)
4,4,Westport Food Shelf
5,5,Community Lunch Box
6,6,Bellmont Food Pantry
7,7,Bombay Food Pantry


In [8]:
node_dict

{'North_Star_Food_Hub': 0,
 'Bellmont_Food_Pantry': 1,
 'Blue_Mountain_Lake_Hamilton_Helps': 2,
 'Bombay_Food_Pantry': 3,
 'Comfort_Food_Community_Greenwich_Food_Pantry': 4,
 'Minerva_Food_Pantry': 5,
 'Community_Lunch_Box': 6,
 'Westport_Food_Shelf': 7}

In [3]:
node_data

,OrganizationName,Type,ScheduledForDelivery,Organization_Name_Formatted,Address,City,State,Zip,Phone,Organization_Name_For_Matching
Organization_Name_Formatted,,,,,,,,,,
Bellmont_Food_Pantry,Bellmont Food Pantry,Distributor,1,Bellmont_Food_Pantry,9 Hill Rd,Brainardsville,NY,12915,518 425-3461,Bellmont_Food_Pantry
Blue_Mountain_Lake_Hamilton_Helps,Blue Mountain Lake Hamilton Helps,Distributor,1,Blue_Mountain_Lake_Hamilton_Helps,Eagle Nest Rd,Blue Mountain Lak,NY,12812,(518) 352-7391,Blue_Mountain_Lake_Hamilton_Helps
Bombay_Food_Pantry,Bombay Food Pantry,Distributor,1,Bombay_Food_Pantry,50 County Route 4,Bombay,NY,12914,518-521-7627,Bombay_Food_Pantry
North_Star_Food_Hub,North Star Food Hub,Hub,0,North_Star_Food_Hub,"4012 Center St,",Lyons Falls,NY,13368,(315) 741-5872,North_Star_Food_Hub
Comfort_Food_Community_Greenwich_Food_Pantry,Comfort Food Community (Greenwich Food Pantry),Distributor,1,Comfort_Food_Community_Greenwich_Food_Pantry,2530 Route 40,Greenwich,NY,12834,518.538.3522,Comfort_Food_Community_Greenwich_Food_Pantry
Minerva_Food_Pantry,Minerva Food Pantry,Distributor,1,Minerva_Food_Pantry,1652 State Route 28N,Minerva,NY,12851,518-251-2869,Minerva_Food_Pantry
Community_Lunch_Box,Community Lunch Box,Distributor,1,Community_Lunch_Box,136 Main St.,Saranac Lake,NY,12983,518-891-3605,Community_Lunch_Box
Westport_Food_Shelf,Westport Food Shelf,Distributor,1,Westport_Food_Shelf,6605 Main St.,Westport,NY,12993,518-962-4448,Westport_Food_Shelf


In [142]:
node_dict

{'North_Star_Food_Hub': 0,
 'Bellmont_Food_Pantry': 1,
 'Blue_Mountain_Lake_Hamilton_Helps': 2,
 'Bombay_Food_Pantry': 3,
 'Comfort_Food_Community_Greenwich_Food_Pantry': 4,
 'Minerva_Food_Pantry': 5,
 'Community_Lunch_Box': 6,
 'Westport_Food_Shelf': 7}

In [16]:
my_distances_list

[(1, 2, 106.4),
 (1, 3, 30.3),
 (1, 0, 138.0),
 (1, 4, 165.0),
 (1, 5, 120.0),
 (1, 6, 48.9),
 (1, 7, 74.3),
 (2, 3, 107.1),
 (2, 0, 61.0),
 (2, 4, 86.2),
 (2, 5, 36.8),
 (2, 6, 54.2),
 (2, 7, 75.8),
 (3, 0, 124.0),
 (3, 4, 192.0),
 (3, 5, 127.0),
 (3, 6, 61.0),
 (3, 7, 101.0),
 (0, 4, 143.0),
 (0, 5, 99.3),
 (0, 6, 123.0),
 (0, 7, 138.0),
 (4, 5, 64.5),
 (4, 6, 120.0),
 (4, 7, 98.4),
 (5, 6, 75.5),
 (5, 7, 53.2),
 (6, 7, 43.0)]